# Agents

Agents are composed of body parts and sensors.
We will use an already implemented agent composed of a Base and a Head.

We are going to attach sensors to this agent.

In [ ]:
from simple_playgrounds.playgrounds import SingleRoom
from simple_playgrounds import Engine
from simple_playgrounds.agents.controllers import Keyboard
from simple_playgrounds.agents.agents import BaseAgent, HeadAgent

# to display the sensors
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

def plt_image(img):
    plt.axis('off')
    plt.imshow(img)
    plt.show()
    

In [ ]:
import math
my_playground = SingleRoom(size=(200, 200))
my_agent = HeadAgent( controller=Keyboard())

my_playground.add_agent(my_agent)

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, playground= my_playground, screen=False)

plt_image(engine.generate_topdown_image(mode='plt'))

## Adding scene elements 

Now lets add some scene element to the playground, that the agent will be able to perceive.

In [ ]:
from simple_playgrounds.playgrounds.scene_elements import Basic

circular_object = Basic((170, 30, 0.5),default_config_key='pentagon', graspable=True, mass = 10)
my_playground.add_scene_element(circular_object)

circular_object = Basic((170, 160, -0.5), physical_shape='circle', radius=10, texture = [120, 0, 230])
my_playground.add_scene_element(circular_object)

plt_image(engine.generate_topdown_image(mode='plt'))

## Adding sensors to the agent 

Ok Great! Now we will add 2 rgb sensors: one attached to the head, one attached to the body.

Note that the properties of the sensors can be changed with keyword arguments.

Move the agent with keyboard arrows, and its head with `n` and `m` and terminate with `q`.

The animated window will appear outside of the notebook.

In [ ]:
from simple_playgrounds.agents.sensors import RgbCamera

my_agent.add_sensor(RgbCamera(my_agent.head, invisible_elements=my_agent.parts, fov=180, range=150))
# my_agent.add_sensor(RgbCamera(my_agent.base_platform, invisible_elements=my_agent.parts, fov=270, range=150))

plt_image(engine.generate_topdown_image(mode='plt'))

engine = Engine(time_limit=1000000, playground= my_playground, screen=True)

engine.update_observations()
img_sensor = engine.generate_sensor_image(my_agent, mode='plt')
plt_image(img_sensor)
# 
while engine.game_on:

    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_commands()

    reset, terminate = engine.step(actions)
    engine.update_observations()

    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
    if reset:
        engine.reset()
    if terminate:
        engine.terminate()

engine.terminate()
cv2.destroyAllWindows()

## Lets add all sensors and move around

We load a test playground.

In [ ]:
from simple_playgrounds.playgrounds.collection.test import Basics
from simple_playgrounds.agents.sensors import RgbCamera, TopdownSensor, Touch, Lidar

my_playground = Basics()
my_agent = BaseAgent(initial_position=(100, 20, 1.15), controller=Keyboard())
my_playground.add_agent(my_agent)

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, playground= my_playground, screen=True)

my_agent.add_sensor(RgbCamera(my_agent.base_platform))
my_agent.add_sensor(Lidar(my_agent.base_platform))
my_agent.add_sensor(Touch(my_agent.base_platform))
my_agent.add_sensor(TopdownSensor(my_agent.base_platform))

plt_image(engine.generate_topdown_image(mode='plt'))

engine.update_observations()
img_sensor = engine.generate_sensor_image(my_agent, mode='plt')

plt.figure(figsize=(20,20))
plt_image(img_sensor)


while engine.game_on:
    
    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_commands()

    reset, terminate = engine.step(actions)
    engine.update_observations()
    
    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
    if reset:
        engine.reset()
    if terminate:
        engine.terminate()
    
engine.terminate()
cv2.destroyAllWindows()


## Funny addition

Sensors are quite generic, so we could in principle add one to a scene element.
It would act as a camera, that the agent has access to.

Here, we add a sensor to an object which is graspable.

In [ ]:
from simple_playgrounds.agents.agents import BaseInteractiveAgent

my_playground = Basics()
my_agent = BaseInteractiveAgent(initial_position=(100, 50, 1.15), controller=Keyboard())
my_playground.add_agent(my_agent)

# we use the option screen=True to use a keyboard controlled agent later on.
engine = Engine(time_limit=1000000, playground= my_playground, screen=True)

circular_object = Basic((150, 30, 1.8), physical_shape='circle', radius=10, texture = [120, 230, 0], graspable=True, mass=5)
my_playground.add_scene_element(circular_object)

camera = RgbCamera(circular_object)
my_agent.add_sensor(camera)

plt_image(engine.generate_topdown_image(mode='plt'))

engine.update_observations()
img_sensor = engine.generate_sensor_image(my_agent, mode='plt')
plt_image(img_sensor)



while engine.game_on:
    
    engine.display_full_scene()

    actions = {}
    for agent in engine.agents:
        actions[agent.name] = agent.controller.generate_commands()

    reset, terminate = engine.step(actions)
    engine.update_observations()
    
    cv2.imshow('sensor', engine.generate_sensor_image(my_agent))
    cv2.waitKey(20)
    
    if reset:
        engine.reset()
    if terminate:
        engine.terminate()
    
engine.terminate()
cv2.destroyAllWindows()


You now see the agent from the point of view of the green object.